In [2]:
import sys
sys.path.append('/home/jovyan/work/Workdir/Utils')
sys.path.append('/home/jovyan/work/Workdir/Models/Multimodal_BEHRT/Early_integration')
sys.path.append('/home/jovyan/work/Workdir/Models/Multimodal_BEHRT/Early_integration/models')

from handle_file import handle_file
from sklearn.model_selection import train_test_split

import os 
import pandas as pd
import re
import nltk
nltk.download('punkt')
import torch
from transformers import AutoTokenizer, AutoModel#ForMaskedLM
from joblib import Parallel, delayed
from tqdm import tqdm
#import seaborn as sns
#import matplotlib.pyplot as plt
from models.BertForClassification import BertForClassification
#from BEHRTForClassification import *
from models.utils import age_vocab, input_vocab, mod_vocab, delay_vocab
from sklearn.decomposition import PCA
#import matplotlib.pyplot as plt
#import seaborn as sn
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score
#import matplotlib.cm as cm
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE
import torch
import plotly.express as px
import pandas as pd
from itertools import chain
from collections import Counter

#import matplotlib
#matplotlib.use('Agg')  # pylint: disable=multiple-statements
#import matplotlib.pyplot as plt
#from matplotlib import widgets
import warnings
warnings.filterwarnings('ignore')

hf = handle_file()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-05-16 09:47:11.793013: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 09:47:12.843526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-05-16 09:47:12.843651: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-05-16 09:47:12.843662: W tensorflow

In [3]:
#pd.set_option('display.max_colwidth', None)
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':14})

In [4]:
import unicodedata

# Dictionary that contains lemma as key and the synonyms as values 
syno_dico = {'':['']}
             
            
            
def get_keys_from_value(d, val):
    # find the key from a given value when the dictionary contains list as values
    for value in d.values():
        if val in value:
            index = list(d.values()).index(value)
            key = list(d.keys())[index]
            break
    return key

            
           
def synonym_merging(token_to_change, synonym_dico = syno_dico ):
    """ Function that will merge synonym into a same word
    params: synonym_dico: dictionary that contain as key the main token with as values all the synonyms
            token_to_change: initial sequence that contains different synonyms 
    return: new token with the synonyms merged into on main token """
   
    if any(token_to_change in val for val in synonym_dico.values()):
       
        key = get_keys_from_value(synonym_dico, token_to_change)
        return key
    else:
        return token_to_change
    
    
def normalize(text):
    
    ## remove accents
    text = str(text)
    normalized = unicodedata.normalize("NFKD", text)
    normalized = "".join([c for c in normalized if not unicodedata.combining(c)])
        
    ## remove white space
    
    normalized = normalized.strip()
    normalized = normalized.lower()
    return normalized 



def clean_text(seq):
    
    if isinstance(seq, list):
        for i in range(len(seq)): 
            seq[i] = normalize(seq[i])
           
            if ('a©' in seq[i]) or ('a ' in seq[i]):
               
                seq[i] = seq[i].replace('a©', 'e')
                seq[i] = seq[i].replace('ã¨', 'e')
                seq[i] = seq[i].replace('a p', 'op')
                seq[i] = seq[i].replace('a r', 'er')
                seq[i] = seq[i].replace('ã©', 'e')

    elif isinstance(seq, str):
        seq = normalize(seq)
        if ('a©' in seq) or ('a ' in seq):
           
            seq= seq.replace('a©', 'e')
            seq = seq.replace('ã¨', 'e')
            seq = seq.replace('ha pital', 'hopital')  ## not working§
            seq = seq.replace('premia r', 'premier')
            seq = seq.replace('ã©', 'e')
      
    return seq

def lower_categories(categories):
    
    cat = list()
    
    for x in categories:
        try:
            cat.append(x.lower())
        except AttributeError:
            pass
    return cat

def get_categorie_and_service_index(seq):
    services, categories = list(), list()
    for i in range(len(seq)):
        if seq[i] == 'service':
            services.append(i)
        elif seq[i] == 'category':
            categories.append(i)
    return services, categories


def plot_PCA(n_components, pca_obj, df, figname):
    fig= plt.figure(figsize=(10,7))
    ax = sn.barplot(
        y=["PCA component {0}".format(i) for i in np.arange(n_components)],
        x = 100.*pca_obj.explained_variance_ratio_, 
        orient = "h",
        #color = my_colors
        
    )
    ax.set_xlim(0,100)
    plt.xlabel("% of explained variance")
    plt.title("PCA with {0} components,(out of {1} features). \nTotal explained {2:.2f}%".format(n_components, df.shape[1], 100*pca_obj.explained_variance_ratio_.sum()))
    
    for i in ax.patches:
        ax.text(i.get_width(), i.get_y()+0.5, '{0:.2f}'.format(i.get_width()))
    
    plt.savefig(figname+'.png')
    return

def reduce_dimension(data, standardize=True, plot_pca_comp=True, n_pca=50, n_tsne=2, PCA_first=True, seed=2):
    #1. PCA 
    if standardize:
        data = StandardScaler().fit_transform(data)
    
    pca = PCA(n_components=n_pca)
    X_PCA = pca.fit_transform(data)
    hf._dump_pkl(X_PCA, 'CLS_embeddings_PCA')
    
    if plot_pca_comp:
        plot_PCA(n_pca, pca, X_PCA, 'PCA_components')
      
    if PCA_first:
        X_tsne = TSNE(n_components=n_tsne, perplexity=30, early_exaggeration=30, n_iter=1000, random_state=seed).fit_transform(X_PCA)
    else:
        X_tsne = TSNE(n_components=n_tsne, perplexity=30, early_exaggeration=30, n_iter=1000, random_state=seed).fit_transform(data)
        
    df_tsne = pd.DataFrame(X_tsne)
    
    return df_tsne

def scatter_plot_tsne(df, title, figsize = (25, 10), color='cluster', size=5, show_legend=True):
    fig = plt.figure(figsize=figsize)
    colors = px.colors.qualitative.Set3+px.colors.qualitative.Light24+px.colors.qualitative.Dark24+px.colors.qualitative.Alphabet+px.colors.qualitative.Prism+px.colors.qualitative.Antique+px.colors.qualitative.Bold+px.colors.qualitative.Vivid
   
    if show_legend:
        fig = px.scatter(df, x=0, y=1, text='text', color=color, opacity=0.9, width=1500, height=1000)
    else:
        fig = px.scatter(df, x=0, y=1, color=color, width=1500, height=1000, opacity=0.9, color_discrete_sequence=colors, labels = color)
        
    fig.update_traces(textposition='top center', textfont_size=12)
    fig.update_traces(marker_size=size)
    fig.update_layout(plot_bgcolor='white')
    fig.update_xaxes(showgrid=False)#, gridwidth=1, gridcolor='Gray')
    fig.update_yaxes(showgrid=False)#, gridwidth=1, gridcolor='#E2E3DE')
    
    fig.update_layout(legend_font_size=9)
   
    fig.show()
 
def flatten_nested_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_nested_list(item))
        else:
            flat_list.append(item)
    return flat_list

def flatten_dict(dict_with_nested_values):
    flat_dict = dict()
    for k , v in dict_with_nested_values.items():
        if isinstance(v, list):
            flat_dict[k] = flatten_nested_list(v)
        else:
            flat_dict[k] = flatten_nested_list(v)
    return flat_dict

def flatten_list(list_of_list):
    return [x for y in list_of_list for x in y]


## Import datasets

In [ ]:
### Embedding

In [5]:
patients_embeddings = hf._load_pkl('/home/jovyan/work/Workdir/Models/Text model/records_embeddins_cpu_sum.pkl')
len(patients_embeddings)

15150

In [ ]:
### Get reports names 

In [6]:
sequences = hf._load_pkl('/home/jovyan/work/Workdir/Models/Multimodal_BEHRT/Early_integration/tasks/data_preprocessed_completed_for_fine_tune.pkl')
sequences.head(1)

,Num_dossier,inputs_normal_range_age,inputs_normal_range_age_and_delays,inputs_normal_range_and_delta_age,inputs_normal_range_and_delta_age_and_delays,inputs_bins_and_delta_age,inputs_bins_and_delta_age_and_delays,delays_basic_and_age,delays_for_twos_and_age,delays_categorized_basic_and_age,...,modalities_basic_and_age_and_delays_preprocessed,modalities_for_twos_and_age_and_delays_preprocessed,delays_basic_and_age_without_st_preprocessed,delays_categorized_basic_and_age_without_st_preprocessed,modalities_basic_and_age_without_st_preprocessed,delays_for_twos_and_age_without_st_preprocessed,delays_categorized_for_twos_and_age_without_st_preprocessed,modalities_for_twos_and_age_without_st_preprocessed,modalities_basic_and_age_and_delays_without_st_preprocessed,modalities_for_twos_and_age_and_delays_without_st_preprocessed
0,028c6922c695871b15102c2d5edc76ab,"[CLS, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[CLS, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[CLS, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[CLS, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[CLS, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[CLS, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[W0, W0, W0, W0, W0, W0, W0, W0, W0, W0, W0, W...",...,"[DEB, age, age, age, grade, grade, grade, node...","[DEB, CA153, CA153, CA153, MONO, MONO, MONO, L...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[W0, W0, W0, W0, W0, W0, W0, W0, W0, W0, W0, W...","[DEB, age, age, age, grade, grade, grade, node...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[W0, W0, W0, W0, W0, W0, W0, W0, W0, W0, W0, W...","[DEB, CA153, CA153, CA153, MONO, MONO, MONO, L...","[DEB, age, age, age, grade, grade, grade, node...","[DEB, CA153, CA153, CA153, MONO, MONO, MONO, L..."


In [7]:
services = hf._load_csv('/home/jovyan/work/Workdir/Models/Multimodal_BEHRT/Early_integration/data extraction/Plots/df_categories_for_services.csv')
categories= list()
for service in services.columns.to_list():
    categories+=list(services[service].unique())
    
categories = lower_categories(categories)

services = [x.lower() for x in services.columns.tolist()] 
del services[0]

In [8]:
categories = clean_text(categories)
services = clean_text(services)

In [9]:
categories = [synonym_merging(x) for x in categories]
services = [synonym_merging(x) for x in services]

In [10]:
indexes = dict()

for i, row in sequences.iterrows():
    indexes[row['Num_dossier']] = get_categorie_and_service_index(row['modalities_basic_and_age'])

In [11]:
cat_services_in_seq = dict()
for k, v in indexes.items():
    idxs = list()
    seq = sequences.loc[sequences['Num_dossier']==k]['inputs_normal_range_age'].values[0]
   
    for serv, cat in zip(v[0], v[1]):
        idxs.append([seq[serv], seq[cat]])
    cat_services_in_seq[k] = idxs

In [11]:
## Plot embedings

In [12]:
flatten_CLS_embeddings = flatten_nested_list(list(patients_embeddings.values()))

In [13]:
len(flatten_CLS_embeddings)

511620

In [17]:
len(flatten_CLS_embeddings[0:1])

1

In [18]:
# Concatenate along a new dimension (n+1)
flatten_values = torch.stack(flatten_CLS_embeddings, dim=0)#.cpu()  # Assuming n is the last dimension
flatten_values.shape

torch.Size([511620, 768])

In [19]:
text = list()
patients = patients_embeddings.keys()
for p in patients:
    text+=cat_services_in_seq[p]
    
len(text)        

511620

In [15]:
import seaborn as sn
import matplotlib.pyplot as plt 

df_tsne = reduce_dimension(flatten_values, standardize=True, plot_pca_comp=False, n_pca=100, n_tsne=2, PCA_first=True, seed=3)
df_tsne['text'] = text
df_tsne.head()

The file has been saved in pkl format in /home/jovyan/work/Workdir/Models/Text model


,0,1,text
0,86.470345,36.308090,"[radio. interv., echographie et mammographie]"
1,32.458714,21.501181,"[consultations, premiere consultation adulte]"
2,-46.332790,27.481298,"[radiodiagnostic, examen radiologique]"
3,-33.896614,57.311214,"[radio-immuno, scintigraphie]"
4,5.638086,57.303661,"[imagerie, echographie]"


In [67]:
hf._dump_pkl(text, 'text_drbertsum')

The file has been saved in pkl format in /home/jovyan/work/Workdir/Models/Text model
